In [234]:
import numpy as np
import librosa

In [235]:
PATH_METADATA = "musicnet_metadata.csv"

PATH_DATA = "musicnet/data/"
PATH_LABELS = "musicnet/labels/"

PATH_TRAIN_DATA = "musicnet/train_data/"
PATH_TRAIN_LABELS = "musicnet/train_labels/"
PATH_TEST_DATA = "musicnet/test_data/"
PATH_TEST_LABELS = "musicnet/test_labels/"

In [236]:
ORIGINAL_SR = 44100
TARGET_SR = 16000
FMIN = librosa.note_to_hz("A0")
FMIN_MIDI_INDEX = librosa.note_to_midi("A0")
N_NOTES = 88
BINS_PER_NOTE = 3
BINS_PER_OCTAVE = 12 * BINS_PER_NOTE
N_BINS = N_NOTES * BINS_PER_NOTE

WINDOW_LENGTH = 2048
HOP_LENGTH = 512

frac_sr = TARGET_SR / ORIGINAL_SR
sample_indexer = frac_sr / HOP_LENGTH

In [237]:
FRAME_DURATION = HOP_LENGTH/TARGET_SR # Frame duration in seconds
FRAMERATE = 1/FRAME_DURATION # Frames per second

In [238]:
import pretty_midi

def piano_roll_to_pretty_midi(piano_roll, fs=100, program=0):
    '''Convert a Piano Roll array into a PrettyMidi object
     with a single instrument.
    Parameters
    ----------
    piano_roll : np.ndarray, shape=(128,frames), dtype=int
        Piano roll of one instrument
    fs : int
        Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    program : int
        The program number of the instrument.
    Returns
    -------
    midi_object : pretty_midi.PrettyMIDI
        A pretty_midi.PrettyMIDI class instance describing
        the piano roll.
    '''
    notes, frames = piano_roll.shape
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=program)

    # pad 1 column of zeros so we can acknowledge inital and ending events
    piano_roll = np.pad(piano_roll, [(0, 0), (1, 1)], 'constant')

    # use changes in velocities to find note on / note off events
    velocity_changes = np.nonzero(np.diff(piano_roll).T)

    # keep track on velocities and note on times
    prev_velocities = np.zeros(notes, dtype=int)
    note_on_time = np.zeros(notes)

    for time, note in zip(*velocity_changes):
        # use time + 1 because of padding above
        velocity = piano_roll[note, time + 1]
        time = time / fs
        if velocity > 0:
            if prev_velocities[note] == 0:
                note_on_time[note] = time
                prev_velocities[note] = velocity
        else:
            pm_note = pretty_midi.Note(
                velocity=prev_velocities[note],
                pitch=note + FMIN_MIDI_INDEX,
                start=note_on_time[note],
                end=time)
            instrument.notes.append(pm_note)
            prev_velocities[note] = 0
    pm.instruments.append(instrument)
    
    return pm

In [239]:
import mido
from mido import Message, MidiFile, MidiTrack

def data_to_mido(data, bpm=60):

    PPQN = FRAMERATE*12 # Pulses per quarter note
    TEMPO = mido.bpm2tempo(bpm) # MIDI Tempo, microseconds per beat
    
    n_notes, n_frames = data.shape

    # pad zeros so we can acknowledge inital and ending events
    data = np.pad(data, [(0, 0), (1, 1)], 'constant')

    mid = MidiFile()
    
    for i in range(n_notes):
        note = i + FMIN_MIDI_INDEX # add the MIDI index
        changes = np.nonzero(np.diff(data[i]))[0] # list with the index where the array changes from 0 to 1 or vice versa
        if len(changes):
            t = round(mido.second2tick(FRAME_DURATION*changes[0], PPQN, TEMPO))
            track = MidiTrack()
            track.append(Message('program_change', program=0, time=t)) # program 0 is Acoustic Grand Piano
            for j in np.diff(changes):
                t = round(mido.second2tick(FRAME_DURATION*j, PPQN, TEMPO))
                track.append(Message('note_on', channel=0, note=note, velocity=64, time=t))
                track.append(Message('note_off', channel=0, note=note, velocity=0, time=0))
            
            mid.tracks.append(track)

    return mid

In [240]:
import midi

def data_to_midi(data, bpm=60):

    SCALER = 4 # Aribtrary value to round up the PPQN and not lose accuracy
    PPQN = FRAMERATE*SCALER # Pulses per quarter note

    pattern = midi.Pattern(resolution=round(PPQN))
    track = midi.Track()
    tempo_event = midi.events.SetTempoEvent()
    tempo_event.set_bpm(bpm)
    track.append(tempo_event)

    note_array = np.zeros(N_NOTES, dtype=int)
    num_pitches, num_frames = data.shape
    track.make_ticks_abs()
    for t in range(num_frames):
        for pitch in range(num_pitches):
            pressed = data[pitch][t]
            if pressed == 1 and note_array[pitch] == 0:
                track.append(midi.NoteOnEvent(tick=t*SCALER, pitch=pitch+FMIN_MIDI_INDEX, velocity=64))
                note_array[pitch] = 1
            elif pressed == 0 and note_array[pitch] == 1:
                track.append(midi.NoteOnEvent(tick=t*SCALER, pitch=pitch+FMIN_MIDI_INDEX, velocity=0))
                note_array[pitch] = 0
    track.append(midi.EndOfTrackEvent(tick=t*SCALER))
    track.make_ticks_rel()

    pattern.append(track)
    return pattern

In [247]:
import midi

def data_to_midi2(data, bpm=60):
    
    SCALER = 4 # Aribtrary value to round up the PPQN and not lose accuracy
    PPQN = FRAMERATE*SCALER # Pulses per quarter note

    pattern = midi.Pattern(resolution=round(PPQN))
    track = midi.Track()
    tempo_event = midi.events.SetTempoEvent()
    tempo_event.set_bpm(bpm)
    track.append(tempo_event)

    num_pitches, num_frames = data.shape
    for pitch in range(num_pitches):
        midi_pitch = pitch + FMIN_MIDI_INDEX # add the MIDI index
        changes = np.nonzero(np.diff(data[pitch]))[0] # list with the index where the array changes from 0 to 1 or vice versa
        if len(changes):
            for j,k in zip(changes[0::2], changes[1::2]):
                track.append(midi.NoteOnEvent(tick=j*SCALER, pitch=midi_pitch, velocity=64))
                track.append(midi.NoteOnEvent(tick=k*SCALER, pitch=midi_pitch, velocity=0))
    track.append(midi.EndOfTrackEvent(tick=num_frames*SCALER))

    pattern.append(track)
    return pattern

In [242]:
filename = "2303"
data = np.load(filename+"_prediction.npy")
bpm = 106 # Beats per minute

In [243]:
a = piano_roll_to_pretty_midi(data)
a.write(filename+'_pretty.mid')

In [244]:
b = data_to_mido(data)
b.save(filename+'_mido.mid')

In [245]:
c = data_to_midi(data)
midi.write_midifile(filename+'_midi1.mid', c)

In [248]:
d = data_to_midi2(data)
midi.write_midifile(filename+'_midi2.mid', d)

AssertionError: 3436